## Root Mean Square Distance Calculations

In [14]:
import mdtraj as md
import itertools

import numpy as np

import pandas as pd

from pathlib import Path

import seaborn as sns

import soursop as ss
from soursop.sstrajectory import SSTrajectory
NA_EXTENSION = ['D5P', 'DPC',  'DPU',  'DPT',  'DPA',  'DPG',  'R5P',  'RPC' , 'RPU',  'RPT',  'RPA',  'RPG']

# importing mean()
from statistics import mean

def Average(lst):
    return mean(lst)

import matplotlib.pyplot as plt

import itertools
colors = ['#747FE3']
colors2 = ['#E37346']
sns.set_palette(sns.color_palette(colors))


Bad key "text.kerning_factor" on line 4 in
/home/alstonj/miniconda3/envs/labconda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [12]:
#define distance cutoff to be 6 ang
dc = 6

In [13]:
"""
We want to build a dictionary that is able to access each full.pdb/full.xtc for each RNA length + NTDRBD. 
We make a mdtraj object for each simulation.
We calculate the pairwise distances for NTD and RNA
Then we make nested dictionaries that contain v that are bound and unbound and correspond to frames that have a
value corresponding to less than our distance cutoff
"""

traj_dict = {}
for i in [25]:
                   
    traj_dict['system_%i'%(i)] = None  
    prefix = '/work/alstonj/supportingdata/2022/cubuk_et_al_2022/data'
    xtcfilename = prefix+ '/equilibrated_traj.xtc'
    pdbfilename = prefix + '/lammps_fixed_traj.pdb'
    MDTD = md.load(xtcfilename, top = pdbfilename)
    traj_dict['system_%i'%(i)] = {}
    traj_dict['system_%i'%(i)]['SST'] = MDTD
    print(i)

    #Group 1 is the RNA nts

    group_1 = [nucleotide + 173 for nucleotide in range(i)]
    #print(group_1)

    #Group 2 is the NTD, this is always 68nts

    group_2 = [aminoacid for aminoacid in range(68)]
    #print(group_2)

    pairs = list(itertools.product(group_1, group_2))
    #len(pairs)

    #compute the pairwise distance for every frame 
    cc = md.compute_contacts(traj_dict['system_%i'%(i)]['SST'], pairs, ignore_nonprotein = False)

    bound = []
    unbound = []

    for a in range(len(traj_dict['system_%i'%(i)]['SST'])):
        if any(a <= dc for a in cc[0][a]):
            bound.append(a)
            if a % 10000 == 0:
                print('bound', a)

        else:
            unbound.append(a)
            if a % 10000 == 0:
                print('unbound', a)


    print(traj_dict['system_%i'%(i)]['SST'])

    traj_dict['system_%i'%(i)]['Bound'] = bound
    traj_dict['system_%i'%(i)]['UnBound'] = unbound
    print(len(bound))
    print(len(unbound))


    bound_df = pd.DataFrame.from_dict(data=traj_dict['system_%i'%(i)]['Bound'])

    bound_df.to_csv(prefix + '/bound_frames.csv', header=False)

    unbound_df = pd.DataFrame.from_dict(data=traj_dict['system_%i'%(i)]['UnBound'])

    unbound_df.to_csv(prefix + '/unbound_frames.csv', header=False)



25
unbound 0
<mdtraj.Trajectory with 2985 frames, 198 atoms, 198 residues, and unitcells>
2330
655


In [15]:
#Replicates = a, for test this is 1

for a in [1]:
    
#RNA Lengths, so you can analyze in batch, but for test this is 25nt
    for i in [25]:
        traj_dict['%i_system_%i'%(i,a)] = {}
        traj_dict['%i_system_%i'%(i,a)]['Bound'] = pd.read_csv(prefix+'/bound_frames.csv', header = None)
        traj_dict['%i_system_%i'%(i,a)]['UnBound'] = pd.read_csv(prefix+'/unbound_frames.csv', header = None)

In [ ]:
#Generate the SST for each RNA:Protein and store in a corresponding dict

for a in [1]:
    for i in [25]:
        #print(i)
        prefix = "/work/alstonj/2022/Phase_Separation/LAMMPS/NTDRBD_PolyrU/NTDRBD_Titrate_Length_NoSalt/%i/%i_300_%i/FULL"%(i,i,a)
        xtcfilename = prefix+ '/full.xtc'
        pdbfilename = prefix + '/full.pdb'
        SST = SSTrajectory(xtcfilename, pdbfilename,extra_valid_residue_names=NA_EXTENSION)
        traj_dict['%i_protein_%i_all'%(i,a)] = {}
        traj_dict['%i_protein_%i_all'%(i,a)]['SST'] = SST
        traj_dict['%i_protein_%i'%(i,a)] = {}
        traj_dict['%i_protein_%i'%(i,a)]['SST'] = SST.proteinTrajectoryList[0]
        traj_dict['%i_rna_%i'%(i,a)] = {}
        traj_dict['%i_rna_%i'%(i,a)]['SST'] = SST.proteinTrajectoryList[1]